In [1]:
import cv2
print((cv2.__version__).split('.'))
import numpy as np
import os
import sys
from random import randint


['3', '4', '1']


# multi tracking

In [2]:
from __future__ import print_function

In [3]:
tracker_type='CSRT'
tracker = cv2.TrackerCSRT_create()
# Set video to load
Path = 'C:\\Users\\Edouard Geze\\Desktop\\Etude\\trimestre 2\\computeur vision\\TP3\\TP3_data(final)\\'

videoPath = Path + "video.mp4"

# Create a video capture object to read videos
cap = cv2.VideoCapture(videoPath)
 
# Read first frame
success, frame = cap.read()



#       cor x cor y ep x ep y
bboxes = [(917, 498, 195, 257), (1217, 455, 190, 242)] #pas mal

colors = [(0, 0, 0), (0, 0, 255)] 

# Create MultiTracker object
multiTracker = cv2.MultiTracker_create()

tracker_type='CSRT'


# Initialize MultiTracker 
for bbox in bboxes:
    multiTracker.add(cv2.TrackerCSRT_create(), frame, bbox)



tab=np.zeros((2024,6),dtype=int)#2022
ligne=0
tab[0]=[ligne+1,1,bboxes[0][0],bboxes[0][2]+bboxes[0][0],bboxes[0][1],bboxes[0][3]+bboxes[0][1]]
tab[1]=[ligne+1,2,bboxes[1][0],bboxes[1][2]+bboxes[1][0],bboxes[1][1],bboxes[1][3]+bboxes[1][1]]
# Process video and track objects
while cap.isOpened():
    ligne=ligne+1
    
    
    
    if ligne ==450:
        bboxes = [(700, 700, 300, 370), (850, 480 , 180 ,220)] 
        colors = [(0, 0, 0), (0, 0, 255)] 
        multiTracker = cv2.MultiTracker_create()
        for bbox in bboxes:
            multiTracker.add(cv2.TrackerCSRT_create(), frame, bbox)
    
    
    
    
    
    success, frame = cap.read()
    if not success:
        break
        
    # Start timer
    timer = cv2.getTickCount()
   
  # get updated location of objects in subsequent frames
    success, boxes = multiTracker.update(frame)
    
    # Calculate Frames per second (FPS)
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
 
        
  # draw tracked objects
    for i, newbox in enumerate(boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(frame, p1, p2, colors[i], 2, 1)
 
    # Display tracker type on frame
    cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);

    # Display FPS on frame
    cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
    
      # show frame
    cv2.namedWindow('MultiTracker', cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty('MultiTracker',cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
    cv2.imshow('MultiTracker', frame)
    tab[ligne*2]=[ligne+1,1,boxes[0][0],boxes[0][2]+boxes[0][0],boxes[0][1],boxes[0][3]+boxes[0][1]]
    tab[ligne*2+1]=[ligne+1,2,boxes[1][0],boxes[1][2]+boxes[1][0],boxes[1][1],boxes[1][3]+boxes[1][1]]
      
    if  ligne == 2022/2:
        break
    
    
    
    # quit on ESC button
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
tab=tab[0:2022]
np.savetxt('test.txt', tab, delimiter=' ', fmt='%i')

# Autres test

# simple tracking

In [4]:
def tracker(bbox):
    
    tracker_type='CSRT'
    tracker =cv2.TrackerCSRT_create()
    # Read video
    video = cv2.VideoCapture("video.avi")


    # Exit if video not opened.
    if not video.isOpened():
        print ("Could not open video")
        sys.exit()

    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print ('Cannot read video file')
        sys.exit()


    # Uncomment the line below to select a different bounding box
    #bbox = cv2.selectROI(frame, False)
    


    # Initialize tracker with first frame and bounding box
    ok = tracker.init(frame, bbox)
    print(bbox)
    bboxpred=np.array(bbox)
    while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break

        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        ok, bbox = tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);

        # Draw bounding box
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);

        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);

        # Display result
        cv2.imshow("Tracking", frame)
        bboxpred=np.append(bboxpred,bbox)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    return bboxpred

# tranform frame into video

In [5]:
def video(Path):
    image_folder = '1'
    video_name = 'video.avi'

    images = [img for img in os.listdir(Path) if img.endswith(".jpg")]
    frame = cv2.imread(os.path.join(Path, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, 0, 30, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(Path, image)))

    cv2.destroyAllWindows()
    video.release()

In [6]:
def trouver_bbox_vrais(Path):
    f = open(Path + 'groundtruth.txt', 'r')
    content = f.read()
    l = content.split('\n')
    l=l[:-1]
    #print(l)
    a=np.array([])
    for i in l:
        a = np.append(a,i.split(','))

    a=np.reshape(a,(int(a.shape[0]/8), 8)).astype(float)
    return a

def score(bbox_vrais,bboxpred):# le score correspond au 
    bboxpred=np.reshape(bboxpred,(int(bboxpred.shape[0]/4), 4)).astype(float)
    bboxpred[:,2]=bboxpred[:,2]+bboxpred[:,0]
    bboxpred[:,3]=bboxpred[:,3]+bboxpred[:,1]
    bbox_vrais[:,3]=bbox_vrais[:,5]
    bbox_vrais[:,4] = (np.minimum(bboxpred[:,2], bbox_vrais[:,2]) - np.maximum(bboxpred[:,0], bbox_vrais[:,0]))*(np.minimum(bboxpred[:,3], bbox_vrais[:,3]) - np.maximum(bboxpred[:,1], bbox_vrais[:,1]))
    bbox_vrais[:,5] =(bboxpred[:,2]-bboxpred[:,0])*(bboxpred[:,3]-bboxpred[:,1])+(bbox_vrais[:,2]-bbox_vrais[:,0])*(bbox_vrais[:,3]-bbox_vrais[:,1])
    bbox_vrais[:,6] = bbox_vrais[:,4]/(bbox_vrais[:,5]-bbox_vrais[:,4])
    score = np.sum(bbox_vrais[:,6])/bbox_vrais[:,6].shape[0]
    return score,bboxpred,bbox_vrais

In [8]:
Path = 'C:\\Users\\Edouard Geze\\Desktop\\Etude\\trimestre 2\\computeur vision\\TP3\\video\\1\\'
bbox_vrais = trouver_bbox_vrais(Path)
bbox=(int(bbox_vrais[0,0]),int(bbox_vrais[0,1]),int(bbox_vrais[0,2]-bbox_vrais[0,0]),int(bbox_vrais[0,5]-bbox_vrais[0,1]))
video(Path)
bboxpred = tracker(bbox)
score,bboxpred,bbox_vrais = score(bbox_vrais,bboxpred)
print(score,)

(496, 419, 40, 42)


TypeError: 'numpy.float64' object is not callable

In [110]:
Path = 'C:\\Users\\Edouard Geze\\Desktop\\Etude\\trimestre 2\\computeur vision\\TP3\\video\\2\\'
bbox_vrais = trouver_bbox_vrais(Path)
bbox=(int(bbox_vrais[0,0]),int(bbox_vrais[0,1]),int(bbox_vrais[0,2]-bbox_vrais[0,0]),int(bbox_vrais[0,5]-bbox_vrais[0,1]))
video(Path)
bboxpred = tracker(bbox)
score,bboxpred,bbox_vrais = score(bbox_vrais,bboxpred)
print(score)

(170, 24, 24, 59)


TypeError: 'numpy.float64' object is not callable

In [111]:
Path = 'C:\\Users\\Edouard Geze\\Desktop\\Etude\\trimestre 2\\computeur vision\\TP3\\video\\3\\'
bbox_vrais = trouver_bbox_vrais(Path)
bbox=(int(bbox_vrais[0,0]),int(bbox_vrais[0,1]),int(bbox_vrais[0,2]-bbox_vrais[0,0]),int(bbox_vrais[0,5]-bbox_vrais[0,1]))
video(Path)
bboxpred = tracker(bbox)
score,bboxpred,bbox_vrais = score(bbox_vrais,bboxpred)
print(score)

(296, 73, 59, 227)


TypeError: 'numpy.float64' object is not callable

In [100]:
Path = 'C:\\Users\\Edouard Geze\\Desktop\\Etude\\trimestre 2\\computeur vision\\TP3\\video\\4\\'
bbox_vrais = trouver_bbox_vrais(Path)
bbox=(int(bbox_vrais[0,0]),int(bbox_vrais[0,1]),int(bbox_vrais[0,2]-bbox_vrais[0,0]),int(bbox_vrais[0,5]-bbox_vrais[0,1]))
video(Path)
bboxpred = tracker(bbox)
score,bboxpred,bbox_vrais = score(bbox_vrais,bboxpred)
print(score)

(277, 82, 26, 55)


TypeError: 'numpy.float64' object is not callable

In [7]:
Path = 'C:\\Users\\Edouard Geze\\Desktop\\Etude\\trimestre 2\\computeur vision\\TP3\\video\\5\\'
bbox_vrais = trouver_bbox_vrais(Path)
bbox=(int(bbox_vrais[0,0]),int(bbox_vrais[0,1]),int(bbox_vrais[0,2]-bbox_vrais[0,0]),int(bbox_vrais[0,5]-bbox_vrais[0,1]))
video(Path)
bboxpred = tracker(bbox)
score,bboxpred,bbox_vrais = score(bbox_vrais,bboxpred)
print(score)

(151, 67, 43, 56)


TypeError: 'numpy.float64' object is not callable

In [ ]:
        #if tracker_type == 'BOOSTING':
       #     tracker = cv2.TrackerBoosting_create()
        if tracker_type == 'MIL':
            tracker = cv2.TrackerMIL_create()
        if tracker_type == 'KCF':
            tracker = cv2.TrackerKCF_create()
        if tracker_type == 'TLD':
            tracker = cv2.TrackerTLD_create()
        if tracker_type == 'MEDIANFLOW':
            tracker = cv2.TrackerMedianFlow_create()
        if tracker_type == 'GOTURN':
            tracker = cv2.TrackerGOTURN_create()
        if tracker_type == 'MOSSE':
            tracker = cv2.TrackerMOSSE_create()
        #if tracker_type == "CSRT":
       #     tracker = cv2.TrackerCSRT_create()

In [ ]:
cap = cv2.VideoCapture(0)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))


while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:

        # write the flipped frame
        out.write(frame)

        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

In [7]:

tracker_type='CSRT'
tracker = cv2.TrackerCSRT_create()
# Read video
video = cv2.VideoCapture("output.avi")


# Exit if video not opened.
if not video.isOpened():
    print ("Could not open video")
    sys.exit()

# Read first frame.
ok, frame = video.read()
if not ok:
    print ('Cannot read video file')
    sys.exit()


# Uncomment the line below to select a different bounding box
bbox = cv2.selectROI(frame, False)



# Initialize tracker with first frame and bounding box
ok = tracker.init(frame, bbox)
print(bbox)
bboxpred=np.array(bbox)
while True:
    # Read a new frame
    ok, frame = video.read()
    if not ok:
        break

    # Start timer
    timer = cv2.getTickCount()

    # Update tracker
    ok, bbox = tracker.update(frame)

    # Calculate Frames per second (FPS)
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);

    # Draw bounding box
    if ok:
        # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
        # Tracking failure
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

    # Display tracker type on frame
    cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);

    # Display FPS on frame
    cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);

    # Display result
    cv2.imshow("Tracking", frame)
    bboxpred=np.append(bboxpred,bbox)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


(370, 287, 48, 56)


In [ ]:
def tracker():
    tracker_type='CSRT'
    tracker = cv2.TrackerCSRT_create()
    # Read video
    video = cv2.VideoCapture("video.avi")


    # Exit if video not opened.
    if not video.isOpened():
        print ("Could not open video")
        sys.exit()

    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print ('Cannot read video file')
        sys.exit()

    # Define an initial bounding box
    #bbox = (287, 23, 86, 320)

    # Uncomment the line below to select a different bounding box
    bbox = cv2.selectROI(frame, False)
    


    # Initialize tracker with first frame and bounding box
    ok = tracker.init(frame, bbox)
    print(bbox)
    bboxpred=np.array(bbox)
    while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break

        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        ok, bbox = tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);

        # Draw bounding box
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);

        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);

        # Display result
        cv2.imshow("Tracking", frame)
        bboxpred=np.append(bboxpred,bbox)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    return bboxpred